In [4]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW                      # 改用 PyTorch 的 AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
from datasets import Dataset as HFDataset
import numpy as np


In [5]:
MODEL_NAME = "bert-base-uncased"
NUM_LABELS = 2            # 根据你的任务改
BATCH_SIZE = 8
EPOCHS = 3
LR = 2e-5
MAX_LEN = 128
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cpu


In [7]:
texts = ["This is a good example.", "This is a bad example.", "Happy product review.", "Terrible experience."]
labels = [1, 0, 1, 0]  # 示例标签
hf_ds = HFDataset.from_dict({"text": texts, "label": labels})
# 划分训练/验证（这里简单切片，按需用 train_test_split）
train_ds = hf_ds.select(range(0, 3))
val_ds = hf_ds.select(range(3, 4))


In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese")

model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-chinese")


/home/szy/APPs/miniconda3/envs/modelscope/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 5478b4f7-6282-43f3-8cd9-b1a26b105750)')' thrown while requesting HEAD https://huggingface.co/google-bert/bert-base-chinese/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: c526137a-e065-4539-985b-2f7e321b8a1f)')' thrown while requesting HEAD https://huggingface.co/google-bert/bert-base-chinese/resolv

KeyboardInterrupt: 

In [5]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
from modelscope import AutoTokenizer, AutoModelForMaskedLM

ModuleNotFoundError: No module named 'addict'

In [ ]:

# 把这里的 model_id 换成你想用的 ModelScope 模型，例如公开的 text-classification 模型 id
# 示例（请替换为可用的 ModelScope 模型 id）:
MODEL_ID = 'google-bert/bert-base-chinese'   # <- 根据 ModelScope Hub 替换
classifier = pipeline(task=Tasks.text_classification, model=MODEL_ID)

# 示例输入
texts = [
    "This is a good example.",
    "This is a bad example.",
    "Happy product review.",
    "Terrible experience."
]

# 批量推理（pipeline 接受单条或多条输入，返回格式依模型而异）
results = classifier(texts)  # 有些 pipeline 预期每次一条，也可以用列表批量
print(results)

# 将 pipeline 输出转为统一的预测与概率（示例，真实字段名视模型返回结构而定）
preds = []
probs = []
for r in results:
    # 可能的格式： {'label': 'positive', 'score': 0.97} 或 [{'label':..., 'score':...}, ...]
    if isinstance(r, list):
        item = r[0]   # 如果返回每条是个列表，取最高的
    else:
        item = r
    preds.append(item.get('label') or item.get('prediction') or None)
    probs.append(item.get('score') or None)

print('preds:', preds)
print('probs:', probs)

skip/download failed for tokenizer_config.json hf_hub_download() got an unexpected keyword argument 'timeout'
skip/download failed for vocab.txt hf_hub_download() got an unexpected keyword argument 'timeout'
skip/download failed for tokenizer.json hf_hub_download() got an unexpected keyword argument 'timeout'


TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-chinese")

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-chinese")

def preprocess(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LEN)

train_ds = train_ds.map(preprocess, batched=True)
val_ds = val_ds.map(preprocess, batched=True)

columns = ["input_ids", "attention_mask", "label"]
train_ds.set_format(type="torch", columns=columns)
val_ds.set_format(type="torch", columns=columns)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: b9032694-4473-4c85-ae69-8c9fbcf95b73)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: bde5adb6-e592-42b2-a533-16d0ed45bc58)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json
Retrying in 2s [Retry 2/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 16de08ca-90e0-4e22-b9ab-3150e8fdf0f9)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json
Retrying in 4s [Retry 3/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout

KeyboardInterrupt: 

In [ ]:
# 模型
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_LABELS)
model.to(DEVICE)

# 优化器与调度器
optimizer = AdamW(model.parameters(), lr=LR)
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1*total_steps), num_training_steps=total_steps)

In [ ]:
# 训练与评估函数
def train_epoch():
    model.train()
    losses = []
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["label"].to(DEVICE)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        losses.append(loss.item())
    return np.mean(losses)

def eval_epoch():
    model.eval()
    preds = []
    truths = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels = batch["label"].to(DEVICE)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            pred = torch.argmax(logits, dim=-1)
            preds.extend(pred.cpu().numpy().tolist())
            truths.extend(labels.cpu().numpy().tolist())
    return accuracy_score(truths, preds)

In [ ]:
# 训练循环
for epoch in range(1, EPOCHS+1):
    loss = train_epoch()
    acc = eval_epoch()
    print(f"Epoch {epoch}  loss={loss:.4f}  val_acc={acc:.4f}")

# 保存模型
save_dir = "./saved-bert-classifier"
os.makedirs(save_dir, exist_ok=True)
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print("Saved to", save_dir)